In [72]:
import pandas as pd
import numpy as np
from string import ascii_lowercase
import itertools
import string
from collections import defaultdict
import random

pd.set_option('display.max_columns', 50)

### Board

In [51]:
connect_n = 4
board_height = 6
board_width = 7 # max 26

In [52]:
columns = [column for column in ascii_lowercase[:board_width]]
rows = [row for row in range(board_height)]

In [53]:
X_cols = []
for column in ascii_lowercase[:board_width]:
    for row in range(board_height):
        X_cols.append(column + str(row))

In [54]:
def new_game(X_cols):
    return pd.Series(0, X_cols)

In [55]:
def empty_positions(rows, columns):
    return {col:[row for row in rows] for col in columns}

### Winning positions

In [56]:
np.random.seed(seed = 1)
X_rand = pd.DataFrame(np.random.randint(3, size=(1000, len(X_cols)), ), columns=X_cols) - 1
win_types = ['up', 'across', 'diag /', 'diag \\']
win_type = 'diag \\'

In [57]:
def rows_and_columns_checked(win_type, all_rows, all_columns):
    win_type_positions = {}
    limit_top_rows = all_rows[:-connect_n + 1]
    limit_bottom_rows = all_rows[connect_n - 1:]
    limit_right_columns =  all_columns[:-connect_n + 1]
    
    # create dictionary of win types and a tuple of the rows and columns checked for each win type
    for win_type in win_types:
        if win_type == 'up':
            rows_checked, columns_checked = limit_top_rows, all_columns
        elif win_type == 'across':
            rows_checked, columns_checked = all_rows, limit_right_columns
        elif win_type == 'diag /':
            rows_checked, columns_checked = limit_top_rows, limit_right_columns
        elif win_type == 'diag \\':
            rows_checked, columns_checked = limit_bottom_rows, limit_right_columns
        else:
            raise ValueError('not a valid win type')
        win_type_positions[win_type] = rows_checked, columns_checked

    # create dictionary of individual row/column positions and a list of their possible win types 
    positions_win_types_to_check = defaultdict(list)
    for win_type, rows_and_columns_to_check in win_type_positions.iteritems():
        for positions in itertools.product(rows_and_columns_to_check[0],rows_and_columns_to_check[1]):
            row = positions[0]
            column = positions[1]
            position = column + str(row)
            positions_win_types_to_check[position, (row, column)].append(win_type)
    return positions_win_types_to_check

In [58]:
def win_type_angle(win_type, row_checked, column_checked):

    # list row and column positions
    if win_type == 'up':
        angle_column_positions = list(column_checked) * connect_n
        angle_row_positions = list(map(str, range(row_checked, row_checked + connect_n)))
    elif win_type == 'across':
        angle_column_positions = list(map(chr, range(ord(column_checked), ord(column_checked) + connect_n)))
        angle_row_positions = list(str(row_checked)) * connect_n
    elif win_type == 'diag /':
        angle_column_positions = list(map(chr, range(ord(column_checked), ord(column_checked) + connect_n)))
        angle_row_positions = list(map(str, range(row_checked, row_checked + connect_n)))
    elif win_type == 'diag \\':
        angle_column_positions = list(map(chr, range(ord(column_checked), ord(column_checked) + connect_n)))
        angle_row_positions = list(map(str, range(row_checked, row_checked - connect_n, -1)))
    else:
        raise ValueError('not a valid win type')

    # combine postitions
    return [column + row for column, row in zip(angle_column_positions, angle_row_positions)]

In [59]:
def get_winning_positions(positions_win_types_to_check):
    winning_positions = defaultdict(list)
    for position, position_win_types in positions_win_types_to_check.iteritems():
        row = position[1][0]
        column = position[1][1]
        for win_type in position_win_types:
            additional_positions_to_check = win_type_angle(win_type, row, column)
            winning_positions[position[0]].append((win_type, additional_positions_to_check))
    return winning_positions

In [60]:
positions_win_types_to_check = rows_and_columns_checked(win_type, rows, columns)

In [61]:
winning_positions = get_winning_positions(positions_win_types_to_check)

### Check for win

In [370]:
def determine_player(game_move_count):
    player_choice = game_move_count % 2
    player_choice *= 2
    player_choice -= 1
    return player_choice

In [267]:
def available_moves(player, remaining_empty_positions):
    return [col + str(rows[0]) for col, rows in remaining_empty_positions.iteritems()]

In [324]:
def remove_empty_position(choosen_move, remaining_empty_positions):
    remaining_empty_positions[choosen_move[0]].pop(0)
    return remaining_empty_positions

In [325]:
def check_positions(game):
    return game[game != 0].index

In [326]:
def obvious_win(game, remaining_empty_positions, winning_positions, player):
    # has_best_move  BOOLEAN
    # move_to_make   STRING board_position OR NULL
    for board_position in check_positions(game):
        for win_type_angle in winning_positions[board_position]:
            game_positions_of_win_type_angle = game[win_type_angle[1]]
            win_type_angle_status = game[win_type_angle[1]].value_counts()
            if len(win_type_angle_status) == 2 and 0 in win_type_angle_status.index and player in win_type_angle_status.index:
                if win_type_angle_status.loc[0] == 1 and win_type_angle_status.loc[player] == 3:
                    winning_move = game_positions_of_win_type_angle[game_positions_of_win_type_angle == 0].index[0]
                    if winning_move in available_moves(player, remaining_empty_positions):
                        return True, winning_move
    return False, None

In [327]:
def predict_best_move(game, remaining_empty_positions, winning_positions, player):
    return random.choice(available_moves(player, remaining_empty_positions))

In [328]:
def choose_move(game, remaining_empty_positions, winning_positions, player):
    # new_board_positions  DICT all board positions
    has_obvious_win = obvious_win(game, remaining_empty_positions, winning_positions, player) 
    if has_obvious_win[0] == True:
        choosen_move = has_obvious_win[1]
        has_won_game = True
    else:
        choosen_move = predict_best_move(game, remaining_empty_positions, winning_positions, player)
        has_won_game = False
    return (choosen_move, has_won_game)

In [329]:
def determine_new_board_positions(game, choosen_move, player):
    game[choosen_move] = player
    return game

In [377]:
random.seed(0)
game = new_game(X_cols)
remaining_empty_positions = empty_positions(rows, columns)
game_move_count = 0

for game_move_count in range(board_height * board_width):
    player = determine_player(game_move_count)
    choosen_move, has_won_game = choose_move(game, remaining_empty_positions, winning_positions, player)
    game = determine_new_board_positions(game, choosen_move, player)
    remaining_empty_positions = remove_empty_position(choosen_move, remaining_empty_positions)
    if has_won_game:
        print player, choosen_move
        print game
        break

-1 f1
a0    0
a1    0
a2    0
a3    0
a4    0
a5    0
b0   -1
b1    1
b2    1
b3    0
b4    0
b5    0
c0    1
c1   -1
c2    1
c3    0
c4    0
c5    0
d0    1
d1   -1
d2    0
d3    0
d4    0
d5    0
e0   -1
e1   -1
e2    1
e3    0
e4    0
e5    0
f0   -1
f1   -1
f2    0
f3    0
f4    0
f5    0
g0   -1
g1    1
g2   -1
g3    1
g4    0
g5    0
dtype: int64


In [176]:
def check_for_win(new_board_positions, player):
    # player    INT 1 or -1 
    # position  STRING board_position
    # win_type  STRING 'up, across, etc.
    
    return (player, position, win_type)

In [177]:
def make_move(player):
    # check for obvious wins
    has_best_move, move_to_make = obvious_win(player)
    if has_best_move == True:
        new_board_positions = make_move(move_to_make, player)
    # evaluate possible next moves
    else:
        move_to_make = evaluate_moves(player)
        new_board_positions = make_move(move_to_make, player)
    # check for win
    return check_for_win(new_board_positions, player)

In [105]:
np.random.seed(seed = 1)
X_rand = pd.DataFrame(np.random.randint(3, size=(1000, len(X_cols)), ), columns=X_cols) - 1

In [106]:
winning_info = {}
for idx, board_positions in X_rand[:10].iterrows():
    winning_info[idx] = 0
    for position, token in board_positions[board_positions.isin([-1, 1])].iteritems():
        for win_type, positions in winning_positions[position]:
            if board_positions[positions].map(lambda x: x == token).all():
                winning_info[idx] = token
                print str(idx) + ' {} | wins in {} position | with a {} win type'.format(token, position, win_type)

1 -1 | wins in b5 position | with a diag \ win type
1 -1 | wins in c4 position | with a diag \ win type
1 -1 | wins in d3 position | with a diag \ win type
2 -1 | wins in d3 position | with a across win type
2 -1 | wins in d5 position | with a diag \ win type
2 -1 | wins in g2 position | with a up win type
3 1 | wins in a5 position | with a diag \ win type
3 1 | wins in a5 position | with a across win type
3 1 | wins in b4 position | with a diag \ win type
5 -1 | wins in a0 position | with a across win type
5 1 | wins in e0 position | with a up win type
5 -1 | wins in f2 position | with a up win type
6 -1 | wins in b1 position | with a up win type
8 1 | wins in a5 position | with a diag \ win type
9 -1 | wins in c0 position | with a across win type
9 1 | wins in f1 position | with a up win type


In [107]:
initial_example = X_rand.loc[6:8]

In [34]:
initial_example.iloc[:2, :3].to_dict()

{'a0': {6: 1, 7: -1}, 'a1': {6: 1, 7: -1}, 'a2': {6: 1, 7: -1}}

In [35]:
dict((k, winning_info[k]) for k in (6, 7, 8))

{6: -1, 7: 0, 8: 1}

In [16]:
winning_info[]

SyntaxError: invalid syntax (<ipython-input-16-c2673684f405>, line 1)

a0    1
a1    1
a2    1
a3   -1
a4    1
a5    1
b0    0
b1   -1
b2   -1
b3   -1
b4   -1
b5    1
c0    0
c1    0
c2   -1
c3    1
c4    1
c5    0
d0    1
d1    0
d2    1
d3    0
d4    1
d5   -1
e0   -1
e1    1
e2    1
e3   -1
e4    0
e5    1
f0   -1
f1    1
f2   -1
f3    0
f4   -1
f5   -1
g0    1
g1   -1
g2    1
g3    0
g4    1
g5   -1
Name: 6, dtype: int64